## 0. 简介

本 note 中主要实现神经网络中常见的函数，所有函数的实现均考虑向量计算，包括：

1. `sigmoid(x)`
2. `relu(x)`
3. `softmax(x)`
4. `cross_entropy_entry(y, t)` `y`是预测标签，`t`是实际数据中标签

## 1. sigmoid

实现如下：

In [3]:
def sigmoid(x):
    '''
    Parameters
    ----------
    x : 任意维度的张量
    
    Return
    ------
    施加 sigmoid 计算后的 x
    '''
    
    return 1 / 1 + np.exp(-x)

## 2. relu

`relu()` 的实现主要是将其行为用 `maximum(0, x)` 函数来实现

In [4]:
def relu(x):
    '''
    Parameters
    ----------
    x : 任意维度的张量
    
    Return
    ------
    施加 relu 计算后的 x
    '''
    return np.maximum(0, x)  # 取 x 和 0 的各个位置上较大的元素返回

## 3. softmax

`softmax` 的实现要点如下：

1. 数值计算的问题，即计算 `exp(a)`时，如果`a` 过大，会出现 `inf` 的情况，两个 `inf` 做除法会出现不确定的情况，而 $C\times \exp(x) = \exp(x + \ln{C})$，所以可以先对 `x` 中的元素先减去对应的最大值避免 `inf` 的情况出现
2. 根据输入数据是向量还是 mini-batch 对应的二维矩阵，分别进行处理

In [5]:
def softmax(x):
    '''
    Paramters
    ---------
    x : 向量或者 mini-batch 下的矩阵，每一行对应一条数据
    
    Return
    ------
    施加 softmax 计算后的 x
    '''
    
    ## 先检测是否是 mini-batch 的情况
    ## mini-batch 需要每一行单独处理
    if x.ndim == 2:
        x = x - x.max(axis=1, keepdims=True) # 各条数据分别减去各自所有元素中的最大值
        x = np.exp(x)
        x /= x.sum(axis=1, keepdims=True)
        
    elif x.ndim == 1:  # 单条数据
        x = x - np.max(x)
        x = np.exp(x) / np.sum(np.exp(x))
        
    return x

## 4. cross_entropy_error

交叉熵误差的实现主要是区分输入数据 `y` 和 `t` 是 one-hot 形式给出还是直接以单独的标签数字给出，可以任意选择以何种格式作为计算方式

另外，为了保证 `log(x)` 中的 $x$ 不接近于 $0$，可以加上一个足够小的值，即 `log(x + 1e-7)`，一般情况下 `1e-7` 会在浮点数的近似表示中被“吸收”

输入数据默认按 mini-batch 处理，其中每一行代表一条数据，因为单条数据属于 mini-batch 的特例

In [ ]:
def corss_entropy_error(y, t):
    '''
    Paramters
    ---------
    y : 预测标签
    t : 正确标签
    
    Return
    ------
    交叉熵误差
    '''
    
    ## 标签数据转 one-hot 格式
    ## 判断是否是标签格式数据
    if t.ndim == 1:  # 标签格式，进行转换
        y = y.reshape(1, )